In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import numpy as np
#from datetime import datetime
#from datetime import date

# See what products have never ever sold

In [2]:
inv = pd.read_csv('InventoryList_2022-02-04.csv',header=0,dtype=str)

inv = inv[['ProductCode','Name','Category','Barcode','PriceTier1','Status']].copy()
inv = inv[inv.Status == 'ACTIVE']
inv

,ProductCode,Name,Category,Barcode,PriceTier1,Status
0,NaN,NaN,Service,NaN,50.0000,ACTIVE
1,777823111113,1 Water 355ml,Non Alcoholic Beverage,777823111113,1.5000,ACTIVE
2,1234500000000,12/24 Mixing Tin Sets,Bar Tools,1234500000000,22.0000,ACTIVE
3,7503013136221,123 Organic Anejo Tequila 750ml,Tequila,7503013136221,59.9900,ACTIVE
4,7503013136207,123 Organic Blanco Tequila 750ml,Tequila,7503013136207,47.9900,ACTIVE
...,...,...,...,...,...,...
6554,9120007600377,Zum Martin Sepp Rose 1L,Rose,9120007600377,14.9900,ACTIVE
6555,9120007600353,Zum Martin Zweigelt,Red,9120007600353,16.9900,ACTIVE
6556,7503018821009,Zunte Mezcal Artesenal 750ml,Mezcal,7503018821009,49.9900,ACTIVE
6557,8437012445055,Zuri Txakolina Blanco 750ml,White,8437012445055,18.9900,ACTIVE


In [3]:
sold = pd.read_excel('2021details.xlsx',engine='openpyxl',header=0,skiprows=5, parse_dates=False)
custs = ['Customer Square ' + str(x) for x in range(1,4)] #we have 3 stores active, we won't count the Warehouse
retail_names = ['Airport','Pleasant Valley','Oak Hill'] # make sure this matches the above list length - no known automation possible

sold = sold[sold.Customer.isin(custs) & sold.Product.notnull()].reset_index(drop=True) #remove wholesale and weird null anomoly entries
sold = sold.drop(['COGS','Profit'], axis=1, inplace=False)#not trusted data columns or useful, we'll ignore them for now

In [4]:
quick = sold.groupby(['Product','SKU','Customer']).Quantity.sum().reset_index()
quick.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10220 entries, 0 to 10219
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Product   10220 non-null  object
 1   SKU       10220 non-null  object
 2   Customer  10220 non-null  object
 3   Quantity  10220 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 319.5+ KB


In [5]:
dead_map = []

for i, row in inv[inv.Status == 'ACTIVE'].iterrows():
    prod = row.Name
    for cust in custs:
        if(quick[(quick.Product == prod) & (quick.Customer == cust)].empty):
            dead_map.append([prod,cust,row.Category])
            
print(dead_map)

[[nan, 'Customer Square 1', 'Service'], [nan, 'Customer Square 2', 'Service'], [nan, 'Customer Square 3', 'Service'], ['1 Water 355ml', 'Customer Square 3', 'Non Alcoholic Beverage'], ['12/24 Mixing Tin Sets', 'Customer Square 1', 'Bar Tools'], ['12/24 Mixing Tin Sets', 'Customer Square 2', 'Bar Tools'], ['12/24 Mixing Tin Sets', 'Customer Square 3', 'Bar Tools'], ['123 Organic Blanco Tequila 750ml', 'Customer Square 3', 'Tequila'], ['1792 12y 750ml', 'Customer Square 2', 'Whiskey Bourbon/American'], ['1792 Small Batch Bourbon 1L', 'Customer Square 1', 'Whiskey Bourbon/American'], ['1792 Small Batch Bourbon 1L', 'Customer Square 3', 'Whiskey Bourbon/American'], ['1792 Sweet Wheat 750ml', 'Customer Square 2', 'Whiskey Bourbon/American'], ['1792 Sweet Wheat 750ml', 'Customer Square 3', 'Whiskey Bourbon/American'], ['1800 Blanco 750ml', 'Customer Square 1', 'Tequila'], ['1800 Blanco 750ml', 'Customer Square 2', 'Tequila'], ['1800 Blanco 750ml', 'Customer Square 3', 'Tequila'], ['1876 Stra

In [6]:
output = pd.DataFrame(dead_map,columns=['Product','Customer','Category'])
output

,Product,Customer,Category
0,NaN,Customer Square 1,Service
1,NaN,Customer Square 2,Service
2,NaN,Customer Square 3,Service
3,1 Water 355ml,Customer Square 3,Non Alcoholic Beverage
4,12/24 Mixing Tin Sets,Customer Square 1,Bar Tools
...,...,...,...
7623,Zunte Mezcal Artesenal 750ml,Customer Square 2,Mezcal
7624,Zunte Mezcal Artesenal 750ml,Customer Square 3,Mezcal
7625,Zuri Txakolina Blanco 750ml,Customer Square 1,White
7626,Zuri Txakolina Blanco 750ml,Customer Square 2,White


In [7]:
output[output.Customer == 'Customer Square 3'].to_csv('NeverSoldOHFULL.csv',index=False)

# Do it again but more smarter, eventually run a longer report

In [32]:
df = pd.read_csv('AvailabilityReport_2022-02-04.csv',header=0,dtype=str)

df = df[['Category','SKU','ProductName','Location','Status','OnHand']].copy()
#df[(df.Status == 'DEPRECATED') & (df.Location == 'Customer Square 3')].head()#good, its empty
df.head()

,Category,SKU,ProductName,Location,Status,OnHand
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,ACTIVE,123.0000
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,ACTIVE,1.0000
2,Non Alcoholic Beverage,777823111113,1 Water 355ml,Pleasant Valley,ACTIVE,5.0000
3,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Airport,ACTIVE,0.0000
4,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Pleasant Valley,ACTIVE,0.0000


In [33]:
sale = pd.read_excel('last365.xlsx',engine='openpyxl',header=0,skiprows=5, parse_dates=False)#sales by product summary last 375 no totals export
sale = sale.drop(['Unit','Invoice','Sale','COGS','Profit'],axis=1).reset_index()
sale = sale[sale.Customer.isin(custs)]
sale

,index,Product,Category,Customer,Quantity
0,0,NaN,Service,Customer Square 1,15
1,1,NaN,Service,Customer Square 2,12
2,2,1 Water 355ml,Non Alcoholic Beverage,Customer Square 1,6
3,3,1 Water 355ml,Non Alcoholic Beverage,Customer Square 2,1
5,5,123 Organic Anejo Tequila 750ml,Tequila,Customer Square 1,3
...,...,...,...,...,...
31196,31196,Zucca Rabarbaro 750ml,Amaro,Customer Square 3,6
31201,31201,Zunte Mezcal Artesenal 750ml,Mezcal,Customer Square 1,1
31202,31202,Zwack Liqueur 750ml,Amaro,Customer Square 1,7
31203,31203,Zwack Liqueur 750ml,Amaro,Customer Square 2,7


In [34]:
store_map = dict(zip(retail_names,custs))
store_map['Airport']#test

'Customer Square 1'

In [35]:
dead_map = []

for i, row in df[df.Status == 'ACTIVE'].iterrows():
    if row.Location == 'Warehouse':
        continue
    prod = row.ProductName
    if(sale[(sale.Product == prod) & (sale.Customer == store_map[row.Location])].empty):
        dead_map.append([prod,store_map[row.Location],row.Category,row.OnHand])


In [36]:
smart = pd.DataFrame(dead_map,columns=['Product','Customer','Category','OnHand'])
smart.to_csv('NeverSoldSmartLast365.csv',index=False)